# Presentation Demo: Live Inference

This notebook demonstrates real-time inference on our deployed endpoint for the presentation.

**Team:** AAI-540 Group 9 (Iman Hamdan, Michael Skirvin)  
**Date:** February 2026

In [2]:
# Install required packages (run once)
!pip install -q boto3 python-dotenv

zsh:1: command not found: pip


In [5]:
# =============================================================================
# PRESENTATION DEMO: Real-Time Inference Example
# =============================================================================

import boto3
import json
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment
load_dotenv('.env')

# Initialize SageMaker runtime
runtime = boto3.client('sagemaker-runtime', region_name='us-east-1')

# Endpoint name from repository
ENDPOINT_NAME = os.getenv("ENDPOINT_NAME", "lgb-hdd-failure-endpoint-20260215-185627")

print("=" * 70)
print("🎯 HARD DRIVE FAILURE PREDICTION - LIVE INFERENCE DEMO")
print("=" * 70)
print(f"\nEndpoint: {ENDPOINT_NAME}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Test Case: High-risk drive with multiple warning indicators
test_record = {
    "pct_one_star": 0.18,      # 18% negative reviews
    "pct_two_star": 0.12,      # 12% two-star reviews
    "smart_5_raw": 8,          # 8 reallocated sectors (WARNING!)
    "smart_187_raw": 3,        # 3 reported uncorrectable errors
    "smart_188_raw": 0,        # No command timeouts
    "smart_197_raw": 12,       # 12 current pending sectors (CRITICAL!)
    "smart_198_raw": 5         # 5 offline uncorrectable sectors
}

print("\n" + "=" * 70)
print("📊 INPUT: Drive Telemetry Data")
print("=" * 70)
print(json.dumps(test_record, indent=2))

try:
    # Call the SageMaker endpoint
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType='application/json',
        Body=json.dumps(test_record)
    )
    
    # Parse response
    result = json.loads(response['Body'].read().decode())
    
    print("\n" + "=" * 70)
    print("🎯 MODEL PREDICTION RESPONSE")
    print("=" * 70)
    print(json.dumps(result, indent=2))
    
    # Extract prediction
    failure_prob = result.get('failure_probability', result.get('probability', 0))
    prediction = result.get('prediction', result.get('predicted_class', 0))
    
    print("\n" + "=" * 70)
    print("📈 BUSINESS INTERPRETATION")
    print("=" * 70)
    if prediction == 1:
        print(f"⚠️  STATUS: HIGH RISK - Drive Predicted to FAIL")
        print(f"   Failure Probability: {failure_prob:.1%}")
        print(f"   🔧 RECOMMENDED ACTION: Schedule immediate replacement")
        print(f"   💰 BUSINESS IMPACT: Prevent unplanned downtime & data loss")
    else:
        print(f"✅ STATUS: LOW RISK - Drive Appears Healthy")
        print(f"   Failure Probability: {failure_prob:.1%}")
        print(f"   📋 RECOMMENDED ACTION: Continue normal monitoring")
    print("=" * 70)
    
    print("\n📷 SCREENSHOT THIS OUTPUT FOR YOUR PRESENTATION!")
    
except Exception as e:
    print(f"\n❌ Error invoking endpoint: {e}")
    print("\n💡 Possible Issues:")
    print(f"   1. Endpoint may have been deleted/stopped")
    print(f"   2. Check endpoint status in AWS Console")
    print(f"   3. Verify AWS credentials are valid")

🎯 HARD DRIVE FAILURE PREDICTION - LIVE INFERENCE DEMO

Endpoint: lgb-hdd-failure-endpoint-20260215-185627
Timestamp: 2026-02-20 23:39:25

📊 INPUT: Drive Telemetry Data
{
  "pct_one_star": 0.18,
  "pct_two_star": 0.12,
  "smart_5_raw": 8,
  "smart_187_raw": 3,
  "smart_188_raw": 0,
  "smart_197_raw": 12,
  "smart_198_raw": 5
}

❌ Error invoking endpoint: Unable to locate credentials

💡 Possible Issues:
   1. Endpoint may have been deleted/stopped
   2. Check endpoint status in AWS Console
   3. Verify AWS credentials are valid
